# Analise Exploratoria Inicial

## Bibliotecas necessarias + opções de visualização do Pandas

In [56]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
import warnings

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:,.2f}'.format
warnings.filterwarnings('ignore')

# Definindo uma seed padrao caso necessite de reproducibilidade.
myseed = 484

# Arquivos auxiliares
from aux_files import *

## Recebendo os dados e abrindo em um DataFrame

Primeiramente, decidi por carregar o arquivo JSON em um variavel, para depois poder usar o `pandas.json_normalize()` o que ia me facilitar em nao ter que lidar com multiplas camadas das variaveis desse arquivo JSON. 

**Fonte dos dados**: [Github > alura-voz/Dados/Telco-Customer-Churn.json](https://github.com/sthemonica/alura-voz/blob/main/Dados/Telco-Customer-Churn.json).

In [57]:
# pegando os dados do github e abrindo o json
data_url = "https://github.com/sthemonica/alura-voz/blob/main/Dados/Telco-Customer-Churn.json?raw=true"
response = urlopen(data_url)
data_json = json.loads(response.read())

# carregando o dataframe com json_normalize
df = pd.json_normalize(data_json, max_level=2, sep='_')

# removendo o prefixo
df.columns = ['_'.join(cols.split('_')[1:]) if len(cols.split('_')) > 1 else cols for cols in df.columns]

## Exploracao

### Introducao

Mostrando o cabecalho dos dados

In [58]:
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4


Verificando se existem linhas duplicadas

In [59]:
df.duplicated().sum()

0

> Nao existem linhas duplicadas.

Verificando as informacoes do DataFrame

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


> Analisando o `DataFrame.info()`  é possível verificar que:
> * O `DataFrame` é formado de 7267 linhas (do `index` 0 ao 7266) e por 21 colunas.
> * Todas as 21 colunas possuem os 7267 valores `non-null` (não nulos)
> * A coluna `Charges_Total` da qual é esperado que o tipo dos dados sejam numéricos estão como `Dtype: object` (string, texto)

Verificando mais atributos das colunas com um funcao que fiz.
* `nunique` - representa a quanitade de valores unicos
* `isnull` - quantidade de valores nulos (ou `np.nan`)
* `black` - quantidade de valor com valores string vazios ou somente com espacos
* `dtypes` - tipo dos dados da coluna
* `unique` - mostra os valores unicos, caso o numero de unicos seja menor que 5

In [61]:
check_cols = df_check(df)
check_cols.set_index('name', inplace=True)
check_cols

,nunique,isnull,blank,dtypes,unique
name,,,,,
customerID,7267,0,0,object,-
Churn,3,0,224,object,"[No, Yes, ]"
gender,2,0,0,object,"[Female, Male]"
SeniorCitizen,2,0,0,int64,"[0, 1]"
Partner,2,0,0,object,"[Yes, No]"
Dependents,2,0,0,object,"[Yes, No]"
tenure,73,0,0,int64,-
PhoneService,2,0,0,object,"[Yes, No]"
MultipleLines,3,0,0,object,"[No, Yes, No phone service]"


> Analisando atravez desse conjunto, pode-se perceber que as colunas que demandam atencao, sao:
> * `Churn`, que contem dados em branco e é a variavel mais importanto, pois é a variavel alvo.
> * `Charges_Total`, que tambem apresentados dados em branco e que deveria dtype: float64.

### Analisando as Colunas

#### Dicionario de Informacao das Colunas

In [62]:
info_cols = pd.DataFrame(aux_info_colunas.items(), columns=['name', 'info'])
info_cols.set_index('name', inplace=True)
info_cols


,info
name,
customerID,Número de identificação único de cada cliente
Churn,Se o cliente deixou ou não a empresa
gender,Gênero (masculino e feminino)
SeniorCitizen,Informação sobre um cliente ter ou não idade igual ou maior que 65 anos
Partner,Se o cliente possui ou não um parceiro ou parceira
Dependents,Se o cliente possui ou não dependentes
tenure,Meses de contrato do cliente
PhoneService,Assinatura de serviço telefônico
MultipleLines,Assisnatura de mais de uma linha de telefone


#### Investigando *Churn*

In [63]:
info_cols.loc['Churn']

info    Se o cliente deixou ou não a empresa
Name: Churn, dtype: object

In [64]:
df['Churn'].value_counts()

No     5174
Yes    1869
        224
Name: Churn, dtype: int64

In [65]:
df['Churn'].value_counts(normalize=True)*100

No    71.20
Yes   25.72
       3.08
Name: Churn, dtype: float64

> Como ja sabia por analise previa o `Churn` apresenta dados em branco, que representam 224 valores dos 7267. (3.08%). 

> Como nao temos um meio de saber o real valor desse dado a solucao encontrada é remover esses dados do conjunto de dados.

In [66]:
df = df.query('Churn != ""')

In [67]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

#### Investigando *Charges_Total*

Verificando a moda, para ver qual o valor em branco. Supondo que seja um dos valores que mais aparecem

In [68]:
df['Charges_Total'].mode().tolist()

[' ', '20.2']

Selecionando as linahs que aparecem com o `Charges_Total` em branco e as colunas de tempo de contrato e cobrancas mensais, para comparacao.

In [69]:
colunas_numericas = ['tenure', 'Charges_Monthly', 'Charges_Total']

df.query('Charges_Total == " "')[colunas_numericas]

,tenure,Charges_Monthly,Charges_Total
975,0,56.05,
1775,0,20.00,
1955,0,61.90,
2075,0,19.70,
2232,0,20.25,
2308,0,25.35,
2930,0,73.35,
3134,0,25.75,
3203,0,52.55,
4169,0,80.85,


> Pode-se perceber que as colunas que apresentam `Charges_Total` em branco sao as que apresentam `tenure == 0`

Substituindo os valores em branco para 0

In [70]:
df['Charges_Total'] = pd.to_numeric(df['Charges_Total'], errors='coerce')
df['Charges_Total'].fillna(0.0, inplace=True)
df[colunas_numericas].describe()

,tenure,Charges_Monthly,Charges_Total
count,"7,043.00","7,043.00","7,043.00"
mean,32.37,64.76,"2,279.73"
std,24.56,30.09,"2,266.79"
min,0.00,18.25,0.00
25%,9.00,35.50,398.55
50%,29.00,70.35,"1,394.55"
75%,55.00,89.85,"3,786.60"
max,72.00,118.75,"8,684.80"


#### Investigando Colunas Numericas

Conferir se as contas batem

In [71]:
(df['tenure'] * df['Charges_Monthly'] == df['Charges_Total']).sum()/df.shape[0]*100

8.874059349708931

> 8.87% dos dados de `Charges_Total`, conferem com a conta `Charges_Monthly` * `tenure`

> Solucao: A coluna `Charges_Total`, vai ser substituida pela conta `Charges_Monthly` * `tenure`, pois se eu fizer o contrario, nos meses de `tenure` = 0, a gasto mensal vai ser 0 tambem.

In [72]:
df['Charges_Total'] = df['tenure'] * df['Charges_Monthly']

df[colunas_numericas].head()

,tenure,Charges_Monthly,Charges_Total
0,9,65.60,590.40
1,9,59.90,539.10
2,4,73.90,295.60
3,13,98.00,"1,274.00"
4,3,83.90,251.70


#### `SeniorCitizen`

para mandar o padrao, decidi alterar os valores da coluna `SeniorCitizen` para Yes e No, para manter o padrao, mesma sabendo que depois precisarei alterar novamente.

In [73]:
df['SeniorCitizen'].unique()

array([0, 1], dtype=int64)

In [74]:
df['SeniorCitizen'] = df['SeniorCitizen'].map({1: 'Yes', 0: 'No'})

In [77]:
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,No,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,590.40
1,0003-MKNFE,No,Male,No,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,539.10
2,0004-TLHLJ,Yes,Male,No,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,295.60
3,0011-IGKFF,Yes,Male,Yes,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,"1,274.00"
4,0013-EXCHZ,Yes,Female,Yes,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,251.70
